In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [18]:
import pandas as pd

# data = pd.read_csv('../data/creditcard.csv')
data = pd.read_csv('../data/oversampling_data.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.996823,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244200,0
1,-1.996823,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342584,0
2,-1.996802,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.158900,0
3,-1.996802,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.139886,0
4,-1.996781,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073813,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566501,-1.129466,-7.145238,5.383562,-9.679664,7.365853,-7.135087,-3.107715,-11.258176,4.501765,-6.438760,-12.633397,7.673385,-13.954257,0.645999,-15.253431,1.341926,-12.118952,-21.374216,-7.753470,3.539377,0.886165,2.215141,0.250707,0.035547,0.440521,0.187297,0.525468,1.502104,0.287044,-0.171154,1
566502,-1.702343,-4.655386,8.532752,-15.222321,10.198547,-4.197514,-3.299981,-10.610038,4.919960,-5.728580,-11.811876,11.183936,-16.630439,0.258371,-17.566520,-0.470995,-10.209341,-13.895245,-5.029679,1.050051,1.430766,1.974833,0.195956,0.482864,-1.203050,-0.340363,0.639373,1.624504,0.731982,-0.349333,1
566503,0.748914,1.187708,3.089764,-6.134725,5.575777,1.604699,-1.933068,-0.965297,0.292589,-2.962763,-3.891017,4.312109,-4.802467,-1.459676,-10.097988,-0.350808,0.508201,0.237821,1.503925,-1.873095,0.267961,0.107235,-0.915751,-0.058251,0.069976,0.353141,0.063025,0.479362,0.326816,-0.353327,1
566504,-1.718906,-1.630195,3.201962,-4.189262,2.308225,-1.419808,-1.820375,-3.521141,1.537790,-1.185527,-6.203047,5.350961,-7.327920,1.352953,-6.463558,-0.544129,-2.652545,-4.963069,-0.615102,0.374866,0.292891,0.516993,-0.499770,-0.060555,-0.418137,0.100704,0.330817,0.268986,0.119987,-0.332985,1


In [19]:
# Preprocessing
X = data.drop('Class', axis=1)
y = data['Class']

In [20]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Converting to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

X_train_tensor.shape, y_train_tensor.shape

(torch.Size([453204, 30]), torch.Size([453204]))

In [22]:
# Model
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # First hidden layer
        self.fc2 = nn.Linear(64, 32)         # Second hidden layer
        self.fc3 = nn.Linear(32, 1)          # Output layer
        self.sigmoid = nn.Sigmoid()          # Sigmoid activation function
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))          # ReLU activation function
        x = torch.relu(self.fc2(x))          # ReLU activation function
        x = self.sigmoid(self.fc3(x))        # Sigmoid activation function for binary classification
        return x

In [23]:
model = Model(X_train_tensor.shape[1])

In [24]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [28]:
# Training
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze()            # Forward pass and remove the extra dimension
    loss = criterion(outputs, y_train_tensor.float())    # Calculate loss
    loss.backward()                                      # Backward pass
    optimizer.step()                                     # Update weights
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')    

Epoch 1/100, Loss: 0.15875987708568573
Epoch 2/100, Loss: 0.14639078080654144
Epoch 3/100, Loss: 0.13575419783592224
Epoch 4/100, Loss: 0.12843254208564758
Epoch 5/100, Loss: 0.123184435069561
Epoch 6/100, Loss: 0.11907161772251129
Epoch 7/100, Loss: 0.11599080264568329
Epoch 8/100, Loss: 0.11740411072969437
Epoch 9/100, Loss: 0.11352124065160751
Epoch 10/100, Loss: 0.11413726955652237
Epoch 11/100, Loss: 0.11253713816404343
Epoch 12/100, Loss: 0.10957285016775131
Epoch 13/100, Loss: 0.10607617348432541
Epoch 14/100, Loss: 0.10257934778928757
Epoch 15/100, Loss: 0.09953141957521439
Epoch 16/100, Loss: 0.09668033570051193
Epoch 17/100, Loss: 0.09356603026390076
Epoch 18/100, Loss: 0.0900459885597229
Epoch 19/100, Loss: 0.08678417652845383
Epoch 20/100, Loss: 0.08426533639431
Epoch 21/100, Loss: 0.0821559801697731
Epoch 22/100, Loss: 0.0800512358546257
Epoch 23/100, Loss: 0.07775142043828964
Epoch 24/100, Loss: 0.07530784606933594
Epoch 25/100, Loss: 0.07313386350870132
Epoch 26/100, Los

In [29]:
# Evaluation
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor).squeeze()
    y_pred = (y_pred_probs > 0.5).float()         # Convert probabilities to binary predictions


# Evaluate performance
print(f'Accuracy: {accuracy_score(y_test_tensor, y_pred):.4f}')
print('Confusion Matrix:')
print(confusion_matrix(y_test_tensor, y_pred))

Accuracy: 0.9983
Classification Report:
[[56276   187]
 [    0 56839]]
